# Three Agent Types Demo - Microsoft Agent Framework (MAF)

**⚠️ Updated: February 2026 - Using Latest MAF Patterns**

This notebook demonstrates the **three types of agents** available in Microsoft Agent Framework and their different chat history management approaches. **This notebook uses the latest API patterns from MAF** (see `/maf-upstream/python/samples/` for official examples).

## What You'll Learn

1. **Azure AI Agent** (`AzureAIProjectAgentProvider`) - Service-managed sessions stored server-side
2. **Azure OpenAI Chat Completion** (`AzureOpenAIChatClient`) - Client-managed history with full control
3. **Azure OpenAI Responses** (`AzureOpenAIResponsesClient`) - Flexible history (both service and custom)

Each demo shows:
- How to create and configure each agent type **using latest MAF patterns**
- Environment variables required for each
- Multi-turn conversations demonstrating memory/context handling
- When to use each agent type based on your requirements

## Key API Changes (Latest MAF)

✅ **Azure AI**: Now uses `AzureAIProjectAgentProvider` (not `AzureAIClient`)  
✅ **Azure OpenAI**: Now uses `.as_agent()` method (not `.create_agent()`)  
✅ **Session Management**: Now uses `agent.create_session()` (preferred over `AgentThread()`)  
✅ **Storage Control**: New `store=True/False` parameter for message persistence  

## Prerequisites

- Python 3.9+ with packages: `agent-framework`, `agent-framework-azure-ai`, `azure-identity`
- Azure CLI authentication: Run `az login` before executing cells
- Azure AI Foundry project (for Azure AI Agent)
- Azure OpenAI resource (for Chat and Responses agents)

**Note**: Each agent type uses different environment variable names - see setup cell below for details.

## Setup: Load Environment Variables

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load .env file
env_path = Path.cwd() / '.env'
if env_path.exists():
    load_dotenv(env_path)
    print("✅ Loaded .env file")
else:
    print("⚠️  No .env file found - using system environment variables")

# Verify required variables
print("\n=== Azure AI Agent Variables ===")
print(f"AZURE_AI_PROJECT_ENDPOINT: {'✅ Set' if os.getenv('AZURE_AI_PROJECT_ENDPOINT') else '❌ Missing'}")
print(f"AZURE_AI_MODEL_DEPLOYMENT_NAME: {'✅ Set' if os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME') else '❌ Missing'}")

print("\n=== Azure OpenAI Chat Completion Variables ===")
print(f"AZURE_OPENAI_ENDPOINT: {'✅ Set' if os.getenv('AZURE_OPENAI_ENDPOINT') else '❌ Missing'}")
print(f"AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: {'✅ Set' if os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME') else '❌ Missing'}")

print("\n=== Azure OpenAI Responses Variables ===")
print(f"AZURE_OPENAI_ENDPOINT: {'✅ Set' if os.getenv('AZURE_OPENAI_ENDPOINT') else '❌ Missing'}")
print(f"AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME: {'✅ Set' if os.getenv('AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME') else '❌ Missing'}")

print("\n⚠️  Note: Each agent type uses DIFFERENT deployment variable names!")
print("See: AQ-CODE/docs/AGENT_ENV_VARS_EXPLAINED.md for details")

✅ Loaded .env file

=== Azure AI Agent Variables ===
AZURE_AI_PROJECT_ENDPOINT: ✅ Set
AZURE_AI_MODEL_DEPLOYMENT_NAME: ✅ Set

=== Azure OpenAI Chat Completion Variables ===
AZURE_OPENAI_ENDPOINT: ✅ Set
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: ✅ Set

=== Azure OpenAI Responses Variables ===
AZURE_OPENAI_ENDPOINT: ✅ Set
AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME: ✅ Set

⚠️  Note: Each agent type uses DIFFERENT deployment variable names!
See: AQ-CODE/docs/AGENT_ENV_VARS_EXPLAINED.md for details


In [2]:
# Workaround for Azure SDK gzip encoding bug (#2457)
# The SDK receives gzip-compressed responses but tries to decode as UTF-8
# Safe to re-run: only patches once
from azure.core.pipeline.policies import HeadersPolicy

if not getattr(HeadersPolicy, '_gzip_patched', False):
    _original_on_request = HeadersPolicy.on_request

    def _patched_on_request(self, request):
        _original_on_request(self, request)
        request.http_request.headers["Accept-Encoding"] = "identity"

    HeadersPolicy.on_request = _patched_on_request
    HeadersPolicy._gzip_patched = True
    print("✅ Applied gzip encoding workaround (Accept-Encoding: identity)")
else:
    print("✅ Gzip workaround already applied")

✅ Applied gzip encoding workaround (Accept-Encoding: identity)


## 1. Azure AI Agent (`AzureAIProjectAgentProvider`)

**⚠️ API CHANGE**: Previously used `AzureAIClient`, now uses `AzureAIProjectAgentProvider`

**Uses**: Azure AI Agents Service (azure-ai-projects SDK backend)  
**Chat History**: Service-managed OR in-memory (controlled by `store` parameter)  
**Best For**: Production scenarios with Azure AI Foundry infrastructure

**Environment Variables Needed**:
- `AZURE_AI_PROJECT_ENDPOINT`
- `AZURE_AI_MODEL_DEPLOYMENT_NAME`

**New Features in Latest Pattern**:
- `await provider.create_agent()` instead of `.create_agent()`
- `agent.create_session()` for session creation
- `store=True/False` parameter to control message persistence

In [3]:
import asyncio
from agent_framework.azure import AzureAIProjectAgentProvider
from azure.identity.aio import AzureCliCredential

async def demo_azure_ai_agent():
    print("\n=== Azure AI Agent Demo (Latest MAF Pattern) ===")
    
    # Check required environment variables
    if not os.getenv('AZURE_AI_PROJECT_ENDPOINT'):
        print("❌ Missing AZURE_AI_PROJECT_ENDPOINT")
        return
    
    # Latest MAF pattern uses AzureAIProjectAgentProvider
    async with (
        AzureCliCredential() as credential,
        AzureAIProjectAgentProvider(credential=credential) as provider,
    ):
        agent = await provider.create_agent(
            name="AzureAIAgent",
            instructions="You are a helpful assistant. Keep responses concise.",
        )
        
        print("✅ Azure AI Agent created successfully")
        print("📝 Sending query (single-turn, no explicit session)...")
        
        # Single-turn interaction without explicit session
        result = await agent.run("What type of agent are you? Answer in one sentence.")
        print(f"🤖 Response: {result.text}")
        
        print(f"💾 Chat History: Service-managed (ephemeral - no persistent session for single-turn)")
        
        print("\n💡 For persistent conversations, create a session:")
        print("   session = agent.create_session()")
        print("   result = await agent.run(query, session=session)")


await demo_azure_ai_agent()


=== Azure AI Agent Demo (Latest MAF Pattern) ===


AzureAIClient does not support runtime tools or structured_output overrides after agent creation. Use AzureOpenAIResponsesClient instead.


✅ Azure AI Agent created successfully
📝 Sending query (single-turn, no explicit session)...
🤖 Response: I am an AI language model designed to assist with information, tasks, and problem-solving.
💾 Chat History: Service-managed (ephemeral - no persistent session for single-turn)

💡 For persistent conversations, create a session:
   session = agent.create_session()
   result = await agent.run(query, session=session)


## 1b. Azure AI Agent - Explicit Session Management (Advanced Pattern - V2 API)

The previous demo used `agent.run()` which abstracts session management. This demo shows **explicit session management using the V2 API** with `AIProjectClient`.

**What is the V2 API?**

Azure AI Foundry Agents V2 (azure-ai-projects 2.x) introduces:
- ✅ **Agent Versioning**: Each agent configuration is saved as an immutable version
- ✅ **`create_version()` method**: Creates new agent versions
- ✅ **`PromptAgentDefinition`**: Structured agent configuration
- ✅ **Version History**: Track all changes, compare configurations, and rollback
- ✅ **Production Stability**: Immutable versions prevent accidental overwrites

**Two Approaches with V2:**
1. **High-level** (Recommended): Use `provider.create_agent()` or `provider.as_agent()` → then `agent.run()` 
2. **Low-level** (This demo): Manual session/message/run management for maximum control

**Why use this pattern?**
- Production systems requiring external integrations
- Custom logging, monitoring, or debugging workflows
- Explicit session persistence and lifecycle management
- Access to intermediate run states

**What you'll see:**
The code below demonstrates the **high-level approach with explicit session control** using V2 API.

In [4]:

from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
from azure.identity.aio import AzureCliCredential
from agent_framework.azure import AzureAIProjectAgentProvider

async def demo_explicit_thread_management():
    """
    Demonstrates explicit session management using Azure AI Foundry V2 API.
    
    V2 API Features:
    - Agent versioning with create_version()
    - Immutable agent versions for production stability
    - PromptAgentDefinition for agent configuration
    - provider.as_agent() to wrap SDK objects without extra HTTP calls
    """
    print("\n=== Azure AI Agent - Explicit Session Management (V2 API) ===")
    
    if not os.getenv('AZURE_AI_PROJECT_ENDPOINT'):
        print("❌ Missing AZURE_AI_PROJECT_ENDPOINT")
        return
    
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(
            endpoint=os.getenv('AZURE_AI_PROJECT_ENDPOINT'),
            credential=credential
        ) as project_client
    ):
        # Step 1: Create an agent version using V2 API
        print("\n1️⃣ Creating agent version (V2 API)...")
        agent_version_details = await project_client.agents.create_version(
            agent_name="ThreadDemoAgent",
            definition=PromptAgentDefinition(
                model=os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME', 'gpt-4o'),
                instructions="You are a helpful assistant demonstrating session management."
            ),
            description="Demo agent for explicit session management (V2 API)"
        )
        print(f"   ✅ Agent created: {agent_version_details.name} (version {agent_version_details.version})")
        print(f"   📦 V2 Feature: This version is immutable and referenced as '{agent_version_details.name}:{agent_version_details.version}'")
        
        try:
            # Step 2: Wrap the agent version as Agent using provider
            print("\n2️⃣ Wrapping agent version as Agent...")
            provider = AzureAIProjectAgentProvider(project_client=project_client)
            agent = provider.as_agent(agent_version_details)  # No HTTP call needed!
            print(f"   ✅ Agent created from version (no additional HTTP call)")
            
            # Step 3: Create a session for conversation
            print("\n3️⃣ Creating session for conversation...")
            session = agent.create_session()
            print(f"   ✅ Session created")
            
            # Step 4: First conversation turn
            print("\n4️⃣ First conversation turn...")
            query1 = "What are the key benefits of session-based conversation management?"
            print(f"   User: {query1}")
            result1 = await agent.run(query1, session=session)
            print(f"   🤖 Agent: {result1.text}")
            
            # Step 5: Second turn - demonstrates session persistence
            print("\n5️⃣ Second conversation turn (shows session persistence)...")
            query2 = "Can you summarize the key point in one sentence?"
            print(f"   User: {query2}")
            result2 = await agent.run(query2, session=session)
            print(f"   🤖 Agent: {result2.text}")
            
            # Step 6: Show session details
            print("\n6️⃣ Session details:")
            if session.service_session_id:
                print(f"   💾 Service session ID: {session.service_session_id}")
                print(f"   💡 This session persists on server and can be resumed later")
            else:
                print(f"   💾 Session managed locally")
            
        finally:
            # Cleanup
            print("\n7️⃣ Cleanup...")
            await project_client.agents.delete_version(
                agent_name=agent_version_details.name, 
                agent_version=agent_version_details.version
            )
            print("   ✅ Agent version deleted")
            print(f"   📦 V2 Feature: Deleted immutable version {agent_version_details.version}")

# Run the demo
await demo_explicit_thread_management()



=== Azure AI Agent - Explicit Session Management (V2 API) ===

1️⃣ Creating agent version (V2 API)...


AzureAIClient does not support runtime tools or structured_output overrides after agent creation. Use AzureOpenAIResponsesClient instead.


   ✅ Agent created: ThreadDemoAgent (version 2)
   📦 V2 Feature: This version is immutable and referenced as 'ThreadDemoAgent:2'

2️⃣ Wrapping agent version as Agent...
   ✅ Agent created from version (no additional HTTP call)

3️⃣ Creating session for conversation...
   ✅ Session created

4️⃣ First conversation turn...
   User: What are the key benefits of session-based conversation management?


AzureAIClient does not support runtime tools or structured_output overrides after agent creation. Use AzureOpenAIResponsesClient instead.


   🤖 Agent: Session-based conversation management offers several key benefits, especially in the context of chatbots, virtual assistants, and online customer service. Here are the main advantages:

### 1. **Context Retention**
- **Consistent Interactions:** Sessions allow the system to remember previous messages and actions. This enables more coherent conversations and less repetition.
- **Personalization:** The assistant can refer to earlier information, making interactions more relevant and tailored to the user.

### 2. **Efficient Problem Solving**
- **Multi-step Tasks:** The session can track progress through complex processes or workflows (e.g., booking appointments, troubleshooting) without the user having to repeat themselves.
- **Error Handling:** If an error occurs, the session helps the system pick up where it left off, rather than starting over.

### 3. **Improved User Experience**
- **Reduced Frustration:** Users don’t need to restate information already provided, which str

### Demo 1b: V2 API with Explicit Session Management

This demo shows how to use the **Azure AI V2 API** (`azure-ai-projects` 2.x) with explicit session management:

**Key V2 API Patterns:**
1. **`create_version()`** - Creates immutable agent versions (not ephemeral agents)
2. **`provider.as_agent()`** - Wraps SDK objects as Agent without HTTP calls  
3. **`agent.create_session()`** - Creates sessions for conversation state
4. **`agent.run(query, session=session)`** - Executes with explicit session control
5. **`delete_version()`** - Removes specific agent versions

**Benefits of V2 API:**
- ✅ Immutable versions prevent accidental changes
- ✅ Version history for rollback and comparison
- ✅ Production-ready agent management
- ✅ Efficient wrapping without extra API calls

**When to use this pattern:**
- Production deployments requiring version control
- Multi-environment scenarios (dev/staging/prod)
- Audit trails of agent configuration changes
- Rollback capabilities for agent updates

Run the code above to see V2 API in action!

## 2. Azure OpenAI Chat Completion (`AzureOpenAIChatClient`)

**⚠️ API CHANGE**: Previously used `.create_agent()`, now uses `.as_agent()`

**Uses**: Azure OpenAI Chat Completion API  
**Chat History**: Custom/client-managed only (you control storage)  
**Best For**: Custom storage requirements, full control over conversation state

**Environment Variables Needed**:
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME` ⚠️ Note: **CHAT**_DEPLOYMENT
- `AZURE_OPENAI_API_VERSION` (optional)

**New Features in Latest Pattern**:
- `.as_agent()` method instead of `.create_agent()`
- `agent.create_session()` for session creation

In [5]:
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

async def demo_azure_openai_chat():
    print("\n=== Azure OpenAI Chat Completion Agent Demo (Latest MAF Pattern) ===")
    
    # Check required environment variables
    if not os.getenv('AZURE_OPENAI_ENDPOINT'):
        print("❌ Missing AZURE_OPENAI_ENDPOINT")
        return
    if not os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME'):
        print("❌ Missing AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
        print("💡 This is different from AZURE_OPENAI_DEPLOYMENT_NAME!")
        return
    
    # Latest MAF pattern uses .as_agent() instead of .create_agent()
    agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
        instructions="You are a helpful assistant. Keep responses concise.",
    )
    
    print("✅ Azure OpenAI Chat Completion Agent created successfully")
    print("📝 Sending query...")
    
    # Note: No session needed for single-turn interactions
    # For multi-turn conversations, use: session = agent.create_session()
    result = await agent.run("What type of agent are you? Answer in one sentence.")
    print(f"🤖 Response: {result.text}")
    print(f"💾 Chat History: Client-managed (you must implement storage)")


# Run the demo
await demo_azure_openai_chat()


=== Azure OpenAI Chat Completion Agent Demo (Latest MAF Pattern) ===
✅ Azure OpenAI Chat Completion Agent created successfully
📝 Sending query...
🤖 Response: I am a conversational AI assistant designed to provide helpful, concise, and accurate responses to your questions.
💾 Chat History: Client-managed (you must implement storage)


## 3. Azure OpenAI Responses (`AzureOpenAIResponsesClient`)

**⚠️ API CHANGE**: Previously used `.create_agent()`, now uses `.as_agent()`

**Uses**: Azure OpenAI Responses API  
**Chat History**: Both service-managed AND custom (most flexible!)  
**Best For**: Scenarios requiring both convenience and customization

**Environment Variables Needed**:
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME` ⚠️ Note: **RESPONSES**_DEPLOYMENT
- `AZURE_OPENAI_API_VERSION` (optional)

**New Features in Latest Pattern**:
- `.as_agent()` method instead of `.create_agent()`
- `agent.create_session()` for session creation

In [6]:
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity import AzureCliCredential

async def demo_azure_openai_responses():
    print("\n=== Azure OpenAI Responses Agent Demo (Latest MAF Pattern) ===")
    
    # Check required environment variables
    if not os.getenv('AZURE_OPENAI_ENDPOINT'):
        print("❌ Missing AZURE_OPENAI_ENDPOINT")
        return
    
    # Check if endpoint is Azure OpenAI (not Azure AI Foundry)
    endpoint = os.getenv('AZURE_OPENAI_ENDPOINT', '')
    if not endpoint.endswith('.openai.azure.com/') and not endpoint.endswith('.openai.azure.com'):
        print("⚠️  Azure OpenAI Responses API is NOT available on Azure AI Foundry endpoints")
        print(f"   Your endpoint: {endpoint}")
        print("   Required: https://your-resource.openai.azure.com/")
        print("\n💡 The Responses API only works with native Azure OpenAI Service endpoints.")
        print("💡 Since you're using Azure AI Foundry, this demo will be skipped.")
        print("💡 Use AzureAIProjectAgentProvider (Demo #1) or AzureOpenAIChatClient (Demo #2) instead.")
        return
    
    if not os.getenv('AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME'):
        print("❌ Missing AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME")
        print("💡 This is different from AZURE_OPENAI_DEPLOYMENT_NAME!")
        print("💡 Set: AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME=your-deployment-name")
        return
    
    try:
        # Latest MAF pattern uses .as_agent() instead of .create_agent()
        agent = AzureOpenAIResponsesClient(credential=AzureCliCredential()).as_agent(
            instructions="You are a helpful assistant. Keep responses concise.",
        )
        
        print("✅ Azure OpenAI Responses Agent created successfully")
        print("📝 Sending query...")
        
        # Note: No session needed for single-turn interactions
        # For multi-turn conversations, use: session = agent.create_session()
        result = await agent.run("What type of agent are you? Answer in one sentence.")
        print(f"🤖 Response: {result.text}")
        print(f"💾 Chat History: Service-managed OR custom (your choice!)")
    except Exception as e:
        print(f"❌ Error: {e}")
        print("\n💡 This typically means the Responses API is not available on your endpoint.")


# Run the demo
await demo_azure_openai_responses()


=== Azure OpenAI Responses Agent Demo (Latest MAF Pattern) ===
✅ Azure OpenAI Responses Agent created successfully
📝 Sending query...
🤖 Response: I am an AI-based conversational agent designed to assist with information, tasks, and problem-solving.
💾 Chat History: Service-managed OR custom (your choice!)


## Comparison: Chat History Management

Let's demonstrate multi-turn conversations with each agent type using the **latest MAF patterns**:

**Key Pattern Changes**:
- Azure AI: Uses `AzureAIProjectAgentProvider` + `await provider.create_agent()`
- Azure OpenAI: Uses `.as_agent()` instead of `.create_agent()`
- Session Management: Uses `agent.create_session()` (preferred over `AgentThread()`)
- Storage Control: New `store=True/False` parameter for Azure AI agents

In [7]:
async def demo_multi_turn_conversations():
    print("\n=== Multi-Turn Conversation Comparison (Latest MAF Patterns) ===")
    
    # Import async credential
    from azure.identity.aio import AzureCliCredential as AsyncAzureCliCredential
    from azure.identity import AzureCliCredential
    
    # Azure AI Agent - Service-managed history
    if os.getenv('AZURE_AI_PROJECT_ENDPOINT'):
        print("\n1️⃣ Azure AI Agent (Service-managed history):")
        
        async with (
            AsyncAzureCliCredential() as credential,
            AzureAIProjectAgentProvider(credential=credential) as provider,
        ):
            agent = await provider.create_agent(
                name="MultiTurnAgent",
                instructions="Remember the conversation context.",
            )
            
            # Use agent.create_session() for conversation state
            # store=False means messages stored in-memory (not on server)
            session = agent.create_session()
            
            # First turn
            result1 = await agent.run("My favorite color is blue.", session=session, store=False)
            print(f"   User: My favorite color is blue.")
            print(f"   Agent: {result1.text}")
            
            # Second turn (agent should remember because we're using the same session)
            result2 = await agent.run("What's my favorite color?", session=session, store=False)
            print(f"   User: What's my favorite color?")
            print(f"   Agent: {result2.text}")
            print(f"   ✅ Service remembers context via session object (in-memory)")
    
    # Azure OpenAI Chat - Client-managed history
    if os.getenv('AZURE_OPENAI_ENDPOINT') and os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME'):
        print("\n2️⃣ Azure OpenAI Chat (Client-managed history):")
        
        # Use .as_agent() instead of .create_agent()
        agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
            instructions="Remember the conversation context.",
        )
        
        session = agent.create_session()
        
        result1 = await agent.run("My favorite color is blue.", session=session)
        print(f"   User: My favorite color is blue.")
        print(f"   Agent: {result1.text}")
        
        result2 = await agent.run("What's my favorite color?", session=session)
        print(f"   User: What's my favorite color?")
        print(f"   Agent: {result2.text}")
        print(f"   ✅ Session stores history client-side")
        
    # Azure OpenAI Responses - Flexible history
    if os.getenv('AZURE_OPENAI_ENDPOINT') and os.getenv('AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME'):
        endpoint = os.getenv('AZURE_OPENAI_ENDPOINT', '')
        if endpoint.endswith('.openai.azure.com/') or endpoint.endswith('.openai.azure.com'):
            print("\n3️⃣ Azure OpenAI Responses (Flexible history):")
            
            agent = AzureOpenAIResponsesClient(credential=AzureCliCredential()).as_agent(
                instructions="Remember the conversation context.",
            )
            
            session = agent.create_session()
            
            result1 = await agent.run("My favorite color is blue.", session=session)
            print(f"   User: My favorite color is blue.")
            print(f"   Agent: {result1.text}")
            
            result2 = await agent.run("What's my favorite color?", session=session)
            print(f"   User: What's my favorite color?")
            print(f"   Agent: {result2.text}")
            print(f"   ✅ Can use service OR custom history via session")
        else:
            print("\n3️⃣ Azure OpenAI Responses (Skipped - requires .openai.azure.com endpoint)")

# Run the demo
await demo_multi_turn_conversations()


=== Multi-Turn Conversation Comparison (Latest MAF Patterns) ===

1️⃣ Azure AI Agent (Service-managed history):


AzureAIClient does not support runtime tools or structured_output overrides after agent creation. Use AzureOpenAIResponsesClient instead.
AzureAIClient does not support runtime tools or structured_output overrides after agent creation. Use AzureOpenAIResponsesClient instead.


   User: My favorite color is blue.
   Agent: Got it! Your favorite color is blue. If you’d like, I can personalize recommendations, stories, or responses with this in mind. Let me know how you’d like to use your favorite color!
   User: What's my favorite color?
   Agent: Your favorite color is blue!
   ✅ Service remembers context via session object (in-memory)

2️⃣ Azure OpenAI Chat (Client-managed history):
   User: My favorite color is blue.
   Agent: Got it! Your favorite color is blue. If you ever want recommendations or ideas related to blue, just let me know!
   User: What's my favorite color?
   Agent: Your favorite color is blue!
   ✅ Session stores history client-side

3️⃣ Azure OpenAI Responses (Flexible history):
   User: My favorite color is blue.
   Agent: Got it! I'll remember that your favorite color is blue. If you ever want recommendations or suggestions that include your favorite color, just let me know!
   User: What's my favorite color?
   Agent: Your favorite col

## Summary: When to Use Each Agent Type (Latest MAF)

### ✅ Use Azure AI Agent (`AzureAIProjectAgentProvider`) when:
- You want service-managed conversation sessions
- Building production apps with Azure AI Foundry/Projects
- Need hosted agent infrastructure
- Want both in-memory and server-stored session options (`store` parameter)
- Need access to Azure AI Agent features (code interpreter, file search, etc.)

### ✅ Use Azure OpenAI Chat Completion (`AzureOpenAIChatClient`) when:
- You need full control over chat history storage
- Implementing custom persistence (database, Redis, etc.)
- Want to manage conversation state manually
- Building custom chat applications
- Don't need Azure AI Agent-specific features

### ✅ Use Azure OpenAI Responses (`AzureOpenAIResponsesClient`) when:
- You want the flexibility of both options
- Need structured response generation
- Want to choose between service or custom history
- Building applications that may evolve requirements
- Working with native Azure OpenAI Service endpoints (not Azure AI Foundry)

---

## Complete .env File Template

⚠️ **CRITICAL**: Each agent type uses **different deployment variable names**!

Create a `.env` file in your project root with these variables:

```bash
# ========================================
# Azure AI Agent (AzureAIProjectAgentProvider)
# ========================================
AZURE_AI_PROJECT_ENDPOINT="https://your-ai-services.services.ai.azure.com/api/projects/your-project"
AZURE_AI_MODEL_DEPLOYMENT_NAME="gpt-4o"

# ========================================
# Azure OpenAI Chat Completion (AzureOpenAIChatClient)
# ========================================
AZURE_OPENAI_ENDPOINT="https://your-resource.openai.azure.com/"
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="gpt-4o"        # Note: CHAT_DEPLOYMENT
AZURE_OPENAI_API_VERSION="2024-10-21"

# ========================================
# Azure OpenAI Responses (AzureOpenAIResponsesClient)
# ========================================
# Uses same AZURE_OPENAI_ENDPOINT as above
AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME="gpt-4o"   # Note: RESPONSES_DEPLOYMENT

# Authentication: Run 'az login' in terminal
```

**Important**: 
- Don't commit your `.env` file to git! Add it to `.gitignore`.
- See `AQ-CODE/docs/AGENT_ENV_VARS_EXPLAINED.md` for detailed explanation of variable naming.

---

## 🔄 API Migration Quick Reference

If you have existing code using the old patterns:

```python
# OLD → NEW: Azure AI Agent
from agent_framework.azure import AzureAIClient  # ❌ Old
from agent_framework.azure import AzureAIProjectAgentProvider  # ✅ New

# OLD → NEW: Agent creation
AzureAIClient(...).create_agent(...)  # ❌ Old
AzureAIProjectAgentProvider(...) → await provider.create_agent(...)  # ✅ New

# OLD → NEW: Azure OpenAI agents
client.create_agent(...)  # ❌ Old
client.as_agent(...)      # ✅ New

# OLD → NEW: Session creation
AgentThread()              # ❌ Old
agent.create_session()     # ✅ New

# OLD → NEW: Running with session
agent.run(query, thread=thread)    # ❌ Old
agent.run(query, session=session)  # ✅ New
```

## 🎓 Key Learnings from This Demo

### ⚠️ IMPORTANT: Latest MAF API Changes (Feb 2026)

This notebook has been **updated to use the latest Microsoft Agent Framework patterns**. Key API changes:

#### 1. Azure AI Agent - New Provider Pattern
```python
# ❌ OLD Pattern (pre-Jan 2026)
from agent_framework.azure import AzureAIClient
credential = AzureCliCredential()
async with AzureAIClient(async_credential=credential).create_agent(...) as agent:

# ✅ NEW Pattern (Latest MAF)
from agent_framework.azure import AzureAIProjectAgentProvider
async with AzureAIProjectAgentProvider(credential=credential) as provider:
    agent = await provider.create_agent(...)
```

#### 2. Azure OpenAI Clients - New .as_agent() Method
```python
# ❌ OLD Pattern
agent = AzureOpenAIChatClient(credential=cred).create_agent(...)

# ✅ NEW Pattern
agent = AzureOpenAIChatClient(credential=cred).as_agent(...)
```

#### 3. Session Management - New agent.create_session() Method
```python
# ❌ OLD Pattern
from agent_framework import AgentThread
thread = AgentThread()
result = await agent.run(query, thread=thread)

# ✅ NEW Pattern
session = agent.create_session()
result = await agent.run(query, session=session)
```

#### 4. Server vs In-Memory Storage Control
```python
# NEW: Control where messages are stored
result = await agent.run(query, session=session, store=False)  # In-memory only
result = await agent.run(query, session=session, store=True)   # Server-side storage
result = await agent.run(query, session=session)               # Default behavior
```

---

### Conversation State Management
**Critical Discovery**: Without passing a session object, each `agent.run()` call is completely independent - the agent has NO memory of previous interactions.

```python
# ❌ Wrong - Each call is a new conversation
result1 = await agent.run("My favorite color is blue.")
result2 = await agent.run("What's my favorite color?")  # Agent won't remember!

# ✅ Correct - Use session to maintain conversation state
session = agent.create_session()
result1 = await agent.run("My favorite color is blue.", session=session)
result2 = await agent.run("What's my favorite color?", session=session)  # Agent remembers!
```

### Environment Variable Naming
Each agent type uses **different** deployment variable names:

| Agent Type | Deployment Variable Name |
|------------|-------------------------|
| Azure AI Agent | `AZURE_AI_MODEL_DEPLOYMENT_NAME` |
| Azure OpenAI Chat | `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME` |
| Azure OpenAI Responses | `AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME` |

**Why?** This allows flexibility to use different models for different purposes.

### Azure OpenAI Responses API Endpoint Requirements
The Responses API has strict endpoint requirements:
- ✅ **Works with**: `https://your-resource.openai.azure.com/` (Native Azure OpenAI)
- ❌ **Does NOT work with**: `https://your-resource.services.ai.azure.com/` (Azure AI Foundry)

**Why?** The Responses API is a preview feature only available on native Azure OpenAI Service endpoints.

### Chat History Storage Differences

| Agent Type | History Storage | When to Use |
|------------|----------------|-------------|
| **Azure AI Agent** | Service-managed (server-side) or in-memory | Production apps, want Azure to handle persistence |
| **Azure OpenAI Chat** | Client-managed (you store it) | Custom storage (DB, Redis), full control |
| **Azure OpenAI Responses** | Both options available | Most flexible, can choose per use case |

**Important**: All agents use session objects to maintain conversation state during execution. The `store` parameter controls where messages persist.

### Authentication Pattern
All demos use Azure CLI authentication:
1. Run `az login` in terminal
2. Use `AzureCliCredential()` in code
3. Azure handles authentication automatically

This is the recommended approach for development and testing. Production apps should use Managed Identity or Service Principal authentication.

---

### 📚 Migration Guide Summary

| Component | Old API | New API |
|-----------|---------|---------|
| Azure AI Agent | `AzureAIClient` | `AzureAIProjectAgentProvider` |
| Agent Creation | `.create_agent()` | `await provider.create_agent()` (AI Agent) or `.as_agent()` (OpenAI) |
| Session Creation | `AgentThread()` | `agent.create_session()` |
| Running with State | `thread=thread` | `session=session` |
| Storage Control | N/A | `store=True/False` parameter |

💡 **See upstream samples**: `/maf-upstream/python/samples/getting_started/` for more examples using latest patterns.

## References

- [Microsoft Learn - Agent Types](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-types/?pivots=programming-language-python)
- [MAF GitHub - Azure AI Samples](https://github.com/microsoft/agent-framework/tree/main/python/samples/getting_started/agents/azure_ai)
- [MAF GitHub - Azure OpenAI Samples](https://github.com/microsoft/agent-framework/tree/main/python/samples/getting_started/agents/azure_openai)